<a href="https://colab.research.google.com/github/sincereQK/QK-ML-Data-study/blob/main/stock_LSTM_20250906.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install tensorflow scikit-learn pandas numpy matplotlib pandas-ta

In [1]:
pip install --upgrade pandas-ta

In [4]:
# !pip install pandas-ta

import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model # load_model 추가
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input
import os # 파일 존재 여부 확인을 위해 추가

# 1. 데이터 불러오기 및 전처리
# /content/drive/MyDrive/Colab Notebooks/20230610_20250610_일봉_000050_경방.csv
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/20230610_20250610_일봉_000050_경방.csv')
# df = pd.read_csv('20230610_20250610_일봉_000050_경방.csv')

df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d')
df = df.sort_values('일자').reset_index(drop=True)
df['sma20'] = ta.sma(df['현재가'], length=20)
df['rsi14'] = ta.rsi(df['현재가'], length=14)
df.dropna(subset=['sma20', 'rsi14'], inplace=True)
df = df.reset_index(drop=True)

# --- 2. 데이터 전처리 ---
features = ['시가', '고가', '저가', '현재가', '거래량', 'sma20', 'rsi14']
data = df[features].copy()
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# --- 3. 모델 불러오기 또는 새로 학습하기 ---
model_filename = 'stock_prediction_model.keras'

if os.path.exists(model_filename):
    print(f"저장된 모델 '{model_filename}'을 불러옵니다.")
    model = load_model(model_filename)
else:
    print("저장된 모델이 없습니다. 새로 모델을 학습하고 저장합니다.")
    # 모델 학습 데이터셋 생성
    train_data_len = int(np.ceil(len(scaled_data) * 0.8))
    train_data = scaled_data[0:train_data_len, :]
    x_train, y_train = [], []
    look_back = 60

    for i in range(look_back, len(train_data)):
        x_train.append(train_data[i-look_back:i, :])
        y_train.append(train_data[i, 3])

    x_train, y_train = np.array(x_train), np.array(y_train)

    # LSTM 모델 구축
    model = Sequential([
        Input(shape=(x_train.shape[1], x_train.shape[2])),
        LSTM(units=50, return_sequences=True),
        Dropout(0.2),
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        Dense(units=25),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')

    # 모델 학습
    model.fit(x_train, y_train, batch_size=32, epochs=50, verbose=1)

    # 모델 저장
    model.save(model_filename)

# --- 4. 다음 날 주가 예측 ---
look_back = 60 # 예측 시에도 동일한 look_back 값 필요
last_look_back_days_scaled = scaled_data[-look_back:]
x_predict = np.reshape(last_look_back_days_scaled, (1, look_back, len(features)))
predicted_price_scaled = model.predict(x_predict)

dummy_predict = np.zeros((1, len(features)))
dummy_predict[:, 3] = predicted_price_scaled
predicted_price = scaler.inverse_transform(dummy_predict)[:, 3][0]

# --- 5. 매매 신호 판단 ---
latest_data = df.iloc[-1]
current_price = latest_data['현재가']
current_sma20 = latest_data['sma20']
current_rsi14 = latest_data['rsi14']

print("\n--- 매매 신호 판단 ---")
print(f"현재 가격: {current_price:,.0f}원")
print(f"다음 날 예측 가격: {predicted_price:,.0f}원")
print(f"현재 20일 이동평균선: {current_sma20:,.0f}원")
print(f"현재 RSI14: {current_rsi14:.2f}")
print("--------------------")

def get_trading_signal(predicted, current, sma20, rsi14):
    is_buy_signal = (predicted > current * 1.02 and current > sma20 and rsi14 < 70)
    is_sell_signal = (predicted < current * 0.98)

    if is_buy_signal:
        print("판단: 매수 신호")
        return 1
    elif is_sell_signal:
        print("판단: 매도 신호")
        return 0
    else:
        print("판단: 관망 (매매 신호 없음)")
        return None

signal = get_trading_signal(predicted_price, current_price, current_sma20, current_rsi14)
print(f"\n최종 출력: {signal}")

저장된 모델 'stock_prediction_model.keras'을 불러옵니다.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 839ms/step

--- 매매 신호 판단 ---
현재 가격: 8,000원
다음 날 예측 가격: 7,539원
현재 20일 이동평균선: 7,580원
현재 RSI14: 58.34
--------------------
판단: 매도 신호

최종 출력: 0
